In [1]:
import numpy as np
import pandas as pd
import os

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#Read data
root_filepath = './Data/'
data_filepath = os.listdir(root_filepath)
data_filepath

['bill_of_materials.csv',
 'comp_adaptor.csv',
 'comp_boss.csv',
 'comp_elbow.csv',
 'comp_float.csv',
 'comp_hfl.csv',
 'comp_nut.csv',
 'comp_other.csv',
 'comp_sleeve.csv',
 'comp_straight.csv',
 'comp_tee.csv',
 'comp_threaded.csv',
 'components.csv',
 'specs.csv',
 'test_set.csv',
 'train_set.csv',
 'tube.csv',
 'tube_end_form.csv',
 'type_component.csv',
 'type_connection.csv',
 'type_end_form.csv']

In [3]:
bill_of_materials = pd.read_csv(root_filepath + data_filepath[0])
comp_adaptor = pd.read_csv(root_filepath + data_filepath[1])
comp_boss = pd.read_csv(root_filepath + data_filepath[2])
comp_elbow = pd.read_csv(root_filepath + data_filepath[3])
comp_float = pd.read_csv(root_filepath + data_filepath[4])
comp_hfl = pd.read_csv(root_filepath + data_filepath[5])
comp_nut = pd.read_csv(root_filepath + data_filepath[6])
comp_other = pd.read_csv(root_filepath + data_filepath[7])
comp_sleeve = pd.read_csv(root_filepath + data_filepath[8])
comp_straight = pd.read_csv(root_filepath + data_filepath[9])
comp_tee = pd.read_csv(root_filepath + data_filepath[10])
comp_threaded = pd.read_csv(root_filepath + data_filepath[11])
components = pd.read_csv(root_filepath + data_filepath[12])
specs = pd.read_csv(root_filepath + data_filepath[13])
test_set = pd.read_csv(root_filepath + data_filepath[14])
train_set = pd.read_csv(root_filepath + data_filepath[15])
tube = pd.read_csv(root_filepath + data_filepath[16])
tube_end_form = pd.read_csv(root_filepath + data_filepath[17])
type_component = pd.read_csv(root_filepath + data_filepath[18])
type_connection = pd.read_csv(root_filepath + data_filepath[19])
type_end_form = pd.read_csv(root_filepath + data_filepath[20])

In [4]:
train_set.head()

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,cost
0,TA-00002,S-0066,2013-07-07,0,0,Yes,1,21.905933
1,TA-00002,S-0066,2013-07-07,0,0,Yes,2,12.341214
2,TA-00002,S-0066,2013-07-07,0,0,Yes,5,6.601826
3,TA-00002,S-0066,2013-07-07,0,0,Yes,10,4.687770
4,TA-00002,S-0066,2013-07-07,0,0,Yes,25,3.541561


In [5]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30213 entries, 0 to 30212
Data columns (total 8 columns):
tube_assembly_id      30213 non-null object
supplier              30213 non-null object
quote_date            30213 non-null object
annual_usage          30213 non-null int64
min_order_quantity    30213 non-null int64
bracket_pricing       30213 non-null object
quantity              30213 non-null int64
cost                  30213 non-null float64
dtypes: float64(1), int64(3), object(4)
memory usage: 1.8+ MB


# Combining dataframes

The dataset came as a set of relational tables, so it would be nice if we could combine all of them into one dataframe. In the below figure you can see the complicated relationship between each file in the dataset.

<img src="./Pics/CAT_ER.png" width="50%" height="50%"/>

In summary:
- Each tube_assembly_id represents a tube whose parameters can be found in the Tube CSV file. This file contains information on the tube such as: diameter, length, thickness...
- Each tube_assembly_id is built by 'components' which is specified in the Bill of Materials CSV file. This file tells us what components are needed to build the tube.
- The measurements/parameters of the components are given by the various comp_() CSV files. These files contain information on the components such as: weight, length...

Ultimately, it stands to reason that a tube's price is determined by the components needed to make it. Therefore, we will have to incorporate all of this information into a single dataframe.

### Tube Features

In [6]:
train = pd.merge(train_set, tube, on ='tube_assembly_id')
train = pd.merge(train, bill_of_materials, on = 'tube_assembly_id')
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30213 entries, 0 to 30212
Data columns (total 39 columns):
tube_assembly_id      30213 non-null object
supplier              30213 non-null object
quote_date            30213 non-null object
annual_usage          30213 non-null int64
min_order_quantity    30213 non-null int64
bracket_pricing       30213 non-null object
quantity              30213 non-null int64
cost                  30213 non-null float64
material_id           29984 non-null object
diameter              30213 non-null float64
wall                  30213 non-null float64
length                30213 non-null float64
num_bends             30213 non-null int64
bend_radius           30213 non-null float64
end_a_1x              30213 non-null object
end_a_2x              30213 non-null object
end_x_1x              30213 non-null object
end_x_2x              30213 non-null object
end_a                 30213 non-null object
end_x                 30213 non-null object
num_boss  

One strange thing that pops out is that 'quantity_3' has more values than 'component_id_3'. How can you have a quantity without listing what component you need the quantity for? Lets take a look into these features.

In [7]:
train.loc[train['component_id_3'].isnull() & train['quantity_3'].notnull()][['component_id_3','quantity_3']]

,component_id_3,quantity_3
4886,NaN,1.0
4887,NaN,1.0
4888,NaN,1.0
4889,NaN,1.0
4890,NaN,1.0
4891,NaN,1.0
4892,NaN,1.0
4893,NaN,1.0
4910,NaN,1.0
4911,NaN,1.0


This definitely looks like a mistake, lets fix it by setting 'quantity_3' to NaN for now.

In [8]:
train.loc[train['component_id_3'].isnull() & train['quantity_3'].notnull(), 'quantity_3'] = np.nan

The <a href="https://www.kaggle.com/c/caterpillar-tube-pricing/data">kaggle website</a> tells us that the tube ends (End A or End X) typically have some end connection. Some tubes require tooling for short end straight lengths (end_a_1x, end_a_2x, end_x_1x, end_x_2x).

This means that tubes that are considered 'short end straight lengths' typically require special tooling which usually means they cost more... Either way this is useful information to us and we will begin by exploring these features further.

In [9]:
display(train[['end_a','end_x','end_a_1x','end_a_2x','end_x_1x','end_x_2x']].head())
display(tube_end_form.head())

,end_a,end_x,end_a_1x,end_a_2x,end_x_1x,end_x_2x
0,EF-008,EF-008,N,N,N,N
1,EF-008,EF-008,N,N,N,N
2,EF-008,EF-008,N,N,N,N
3,EF-008,EF-008,N,N,N,N
4,EF-008,EF-008,N,N,N,N


,end_form_id,forming
0,EF-001,Yes
1,EF-002,No
2,EF-003,No
3,EF-004,No
4,EF-005,Yes


So we see that end_a and end_x contain some kind of ID code. I did a quick check and found that these codes relate to the **tube_end_form** CSV file, which contains a column, 'forming' that contains Yes/No values. Lets create two new features: **forming_a** & **forming_x**.

In [10]:
end_form_ids = tube_end_form['end_form_id'].unique()

for end_form_id in end_form_ids:
    train.loc[train['end_a'] == end_form_id, 'forming_a'] = tube_end_form[tube_end_form['end_form_id'] == end_form_id]['forming'].values[0]
    train.loc[train['end_x'] == end_form_id, 'forming_x'] = tube_end_form[tube_end_form['end_form_id'] == end_form_id]['forming'].values[0]
    train.loc[train['end_a'] == 'NONE', 'forming_a'] = 'NONE'
    train.loc[train['end_x'] == 'NONE', 'forming_x'] = 'NONE'

In [11]:
train[['end_a','end_x','forming_a','forming_x','end_a_1x','end_a_2x','end_x_1x','end_x_2x']].head()

,end_a,end_x,forming_a,forming_x,end_a_1x,end_a_2x,end_x_1x,end_x_2x
0,EF-008,EF-008,Yes,Yes,N,N,N,N
1,EF-008,EF-008,Yes,Yes,N,N,N,N
2,EF-008,EF-008,Yes,Yes,N,N,N,N
3,EF-008,EF-008,Yes,Yes,N,N,N,N
4,EF-008,EF-008,Yes,Yes,N,N,N,N


### Component Features

We are going to make features using the multiple component CSV files given in the dataset. First we will create a feature called **component_type_#** that specifies what type of component it is (adaptor, boss, threaded...) and another feature called **component_weight_#** which tells us how much that component weighs.

In [12]:
adaptor_ids = comp_adaptor['component_id'].unique()
boss_ids = comp_boss['component_id'].unique()
elbow_ids = comp_elbow['component_id'].unique()
float_ids = comp_float['component_id'].unique()
hfl_ids = comp_hfl['component_id'].unique()
nut_ids = comp_nut['component_id'].unique()
other_ids = comp_other['component_id'].unique()
sleeve_ids = comp_sleeve['component_id'].unique()
straight_ids = comp_straight['component_id'].unique()
tee_ids = comp_tee['component_id'].unique()
threaded_ids = comp_threaded['component_id'].unique()

#component_id_1
for component_id in train['component_id_1'].unique():
    if component_id in adaptor_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'adaptor'
        weight = comp_adaptor.loc[comp_adaptor['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in boss_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'boss'
        weight = comp_boss.loc[comp_boss['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in elbow_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'elbow'
        weight = comp_elbow.loc[comp_elbow['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in float_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'float'
        weight = comp_float.loc[comp_float['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in hfl_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'hfl'
        weight = comp_hfl.loc[comp_hfl['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in nut_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'nut'
        weight = comp_nut.loc[comp_nut['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in other_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'other'
        weight = comp_other.loc[comp_other['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in sleeve_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'sleeve'
        weight = comp_sleeve.loc[comp_sleeve['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in straight_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'straight'
        weight = comp_straight.loc[comp_straight['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in tee_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'tee'
        weight = comp_tee.loc[comp_tee['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight
    elif component_id in threaded_ids:
        train.loc[train['component_id_1'] == component_id, 'component_type_1'] = 'threaded'
        weight = comp_threaded.loc[comp_threaded['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_1'] == component_id, 'component_weight_1'] = weight

#componend_id_2
for component_id in train['component_id_2'].unique():
    if component_id in adaptor_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'adaptor'
        weight = comp_adaptor.loc[comp_adaptor['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in boss_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'boss'
        weight = comp_boss.loc[comp_boss['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in elbow_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'elbow'
        weight = comp_elbow.loc[comp_elbow['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in float_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'float'
        weight = comp_float.loc[comp_float['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in hfl_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'hfl'
        weight = comp_hfl.loc[comp_hfl['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in nut_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'nut'
        weight = comp_nut.loc[comp_nut['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in other_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'other'
        weight = comp_other.loc[comp_other['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in sleeve_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'sleeve'
        weight = comp_sleeve.loc[comp_sleeve['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in straight_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'straight'
        weight = comp_straight.loc[comp_straight['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in tee_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'tee'
        weight = comp_tee.loc[comp_tee['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight
    elif component_id in threaded_ids:
        train.loc[train['component_id_2'] == component_id, 'component_type_2'] = 'threaded'
        weight = comp_threaded.loc[comp_threaded['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_2'] == component_id, 'component_weight_2'] = weight

#component_id_3
for component_id in train['component_id_3'].unique():
    if component_id in adaptor_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'adaptor'
        weight = comp_adaptor.loc[comp_adaptor['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in boss_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'boss'
        weight = comp_boss.loc[comp_boss['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in elbow_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'elbow'
        weight = comp_elbow.loc[comp_elbow['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in float_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'float'
        weight = comp_float.loc[comp_float['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in hfl_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'hfl'
        weight = comp_hfl.loc[comp_hfl['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in nut_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'nut'
        weight = comp_nut.loc[comp_nut['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in other_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'other'
        weight = comp_other.loc[comp_other['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in sleeve_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'sleeve'
        weight = comp_sleeve.loc[comp_sleeve['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in straight_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'straight'
        weight = comp_straight.loc[comp_straight['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in tee_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'tee'
        weight = comp_tee.loc[comp_tee['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight
    elif component_id in threaded_ids:
        train.loc[train['component_id_3'] == component_id, 'component_type_3'] = 'threaded'
        weight = comp_threaded.loc[comp_threaded['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_3'] == component_id, 'component_weight_3'] = weight

#component_id_4
for component_id in train['component_id_4'].unique():
    if component_id in adaptor_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'adaptor'
        weight = comp_adaptor.loc[comp_adaptor['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in boss_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'boss'
        weight = comp_boss.loc[comp_boss['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in elbow_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'elbow'
        weight = comp_elbow.loc[comp_elbow['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in float_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'float'
        weight = comp_float.loc[comp_float['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in hfl_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'hfl'
        weight = comp_hfl.loc[comp_hfl['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in nut_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'nut'
        weight = comp_nut.loc[comp_nut['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in other_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'other'
        weight = comp_other.loc[comp_other['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in sleeve_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'sleeve'
        weight = comp_sleeve.loc[comp_sleeve['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in straight_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'straight'
        weight = comp_straight.loc[comp_straight['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in tee_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'tee'
        weight = comp_tee.loc[comp_tee['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight
    elif component_id in threaded_ids:
        train.loc[train['component_id_4'] == component_id, 'component_type_4'] = 'threaded'
        weight = comp_threaded.loc[comp_threaded['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_4'] == component_id, 'component_weight_4'] = weight

#component_id_5
for component_id in train['component_id_5'].unique():
    if component_id in adaptor_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'adaptor'
        weight = comp_adaptor.loc[comp_adaptor['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in boss_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'boss'
        weight = comp_boss.loc[comp_boss['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in elbow_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'elbow'
        weight = comp_elbow.loc[comp_elbow['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in float_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'float'
        weight = comp_float.loc[comp_float['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in hfl_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'hfl'
        weight = comp_hfl.loc[comp_hfl['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in nut_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'nut'
        weight = comp_nut.loc[comp_nut['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in other_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'other'
        weight = comp_other.loc[comp_other['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in sleeve_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'sleeve'
        weight = comp_sleeve.loc[comp_sleeve['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in straight_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'straight'
        weight = comp_straight.loc[comp_straight['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in tee_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'tee'
        weight = comp_tee.loc[comp_tee['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight
    elif component_id in threaded_ids:
        train.loc[train['component_id_5'] == component_id, 'component_type_5'] = 'threaded'
        weight = comp_threaded.loc[comp_threaded['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_5'] == component_id, 'component_weight_5'] = weight

#component_id_6
for component_id in train['component_id_6'].unique():
    if component_id in adaptor_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'adaptor'
        weight = comp_adaptor.loc[comp_adaptor['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in boss_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'boss'
        weight = comp_boss.loc[comp_boss['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in elbow_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'elbow'
        weight = comp_elbow.loc[comp_elbow['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in float_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'float'
        weight = comp_float.loc[comp_float['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in hfl_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'hfl'
        weight = comp_hfl.loc[comp_hfl['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in nut_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'nut'
        weight = comp_nut.loc[comp_nut['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in other_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'other'
        weight = comp_other.loc[comp_other['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in sleeve_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'sleeve'
        weight = comp_sleeve.loc[comp_sleeve['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in straight_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'straight'
        weight = comp_straight.loc[comp_straight['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in tee_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'tee'
        weight = comp_tee.loc[comp_tee['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight
    elif component_id in threaded_ids:
        train.loc[train['component_id_6'] == component_id, 'component_type_6'] = 'threaded'
        weight = comp_threaded.loc[comp_threaded['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_6'] == component_id, 'component_weight_6'] = weight

#component_id_7
for component_id in train['component_id_7'].unique():
    if component_id in adaptor_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'adaptor'
        weight = comp_adaptor.loc[comp_adaptor['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in boss_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'boss'
        weight = comp_boss.loc[comp_boss['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in elbow_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'elbow'
        weight = comp_elbow.loc[comp_elbow['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in float_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'float'
        weight = comp_float.loc[comp_float['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in hfl_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'hfl'
        weight = comp_hfl.loc[comp_hfl['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in nut_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'nut'
        weight = comp_nut.loc[comp_nut['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in other_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'other'
        weight = comp_other.loc[comp_other['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in sleeve_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'sleeve'
        weight = comp_sleeve.loc[comp_sleeve['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in straight_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'straight'
        weight = comp_straight.loc[comp_straight['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in tee_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'tee'
        weight = comp_tee.loc[comp_tee['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight
    elif component_id in threaded_ids:
        train.loc[train['component_id_7'] == component_id, 'component_type_7'] = 'threaded'
        weight = comp_threaded.loc[comp_threaded['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_7'] == component_id, 'component_weight_7'] = weight

#component_id_8
for component_id in train['component_id_8'].unique():
    if component_id in adaptor_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'adaptor'
        weight = comp_adaptor.loc[comp_adaptor['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in boss_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'boss'
        weight = comp_boss.loc[comp_boss['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in elbow_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'elbow'
        weight = comp_elbow.loc[comp_elbow['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in float_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'float'
        weight = comp_float.loc[comp_float['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in hfl_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'hfl'
        weight = comp_hfl.loc[comp_hfl['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in nut_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'nut'
        weight = comp_nut.loc[comp_nut['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in other_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'other'
        weight = comp_other.loc[comp_other['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in sleeve_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'sleeve'
        weight = comp_sleeve.loc[comp_sleeve['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in straight_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'straight'
        weight = comp_straight.loc[comp_straight['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in tee_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'tee'
        weight = comp_tee.loc[comp_tee['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight
    elif component_id in threaded_ids:
        train.loc[train['component_id_8'] == component_id, 'component_type_8'] = 'threaded'
        weight = comp_threaded.loc[comp_threaded['component_id'] == component_id, 'weight'].values[0]
        train.loc[train['component_id_8'] == component_id, 'component_weight_8'] = weight

### Date, Month and Year Features

We will break down the feature 'quote_date' into three features: Date, Month and Year.

In [13]:
train['quote_date'] = pd.to_datetime(train['quote_date'])

train['date'] = train['quote_date'].dt.day
train['month'] = train['quote_date'].dt.month
train['year'] = train['quote_date'].dt.year

## Handling Missing Data:

So now that we created some new features. It's time to deal with missing data! There are several things we will be doing:
- If component_id_# or component_type_# is missing a value, means that tube doesn't require any further components. Replace NaN with 'None'
- Similarily with quantity_# and component_weight_#, replace NaN with 0
- Some components do not have their weight values listed in the dataset. We will replace these values with the mean weight of their component type.

In [14]:
#Get Mean Weights for all component types
adaptor_weight = comp_adaptor['weight'].dropna().mean()
boss_weight = comp_boss['weight'].dropna().mean()
elbow_weight = comp_elbow['weight'].dropna().mean()
float_weight = comp_float['weight'].dropna().mean()
hfl_weight = comp_hfl['weight'].dropna().mean()
nut_weight = comp_nut['weight'].dropna().mean()
other_weight = comp_other['weight'].dropna().mean()
sleeve_weight = comp_sleeve['weight'].dropna().mean()
straight_weight = comp_straight['weight'].dropna().mean()
tee_weight = comp_tee['weight'].dropna().mean()
threaded_weight = comp_threaded['weight'].dropna().mean()

#First, find rows with valid component_type but NaN for weight
for i in range(1,9):
    component_type = 'component_type_' + str(i)
    weight = 'component_weight_' + str(i)
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'adaptor'), weight] = adaptor_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'boss'), weight] = boss_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'elbow'), weight] = elbow_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'float'), weight] = float_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'hfl'), weight] = hfl_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'nut'), weight] = nut_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'other'), weight] = other_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'sleeve'), weight] = sleeve_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'straight'), weight] = straight_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'tee'), weight] = tee_weight
    train.loc[train[component_type].notnull() & train[weight].isnull() & (train[component_type] == 'threaded'), weight] = threaded_weight
    
#Fill in the rest of the data
for i in range(1,9):
    component_id = 'component_id_' + str(i)
    component_type = 'component_type_' + str(i)
    weight = 'component_weight_' + str(i)
    quantity = 'quantity_' + str(i)
    train[component_id] = train[component_id].fillna('None')
    train[component_type] = train[component_type].fillna('None')
    train[weight] = train[weight].fillna(0)
    train[quantity] = train[quantity].fillna(0)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30213 entries, 0 to 30212
Data columns (total 60 columns):
tube_assembly_id      30213 non-null object
supplier              30213 non-null object
quote_date            30213 non-null datetime64[ns]
annual_usage          30213 non-null int64
min_order_quantity    30213 non-null int64
bracket_pricing       30213 non-null object
quantity              30213 non-null int64
cost                  30213 non-null float64
material_id           29984 non-null object
diameter              30213 non-null float64
wall                  30213 non-null float64
length                30213 non-null float64
num_bends             30213 non-null int64
bend_radius           30213 non-null float64
end_a_1x              30213 non-null object
end_a_2x              30213 non-null object
end_x_1x              30213 non-null object
end_x_2x              30213 non-null object
end_a                 30213 non-null object
end_x                 30213 non-null object
nu

# Encoding Categorical Values

The last thing to do for data processing is to encode our categorical features. We will be using the one-hot encoding instead of label encoding since the categorical features don't have any relation to each other (adaptor component is not bigger than threaded component).

In [16]:
#One Hot Encode
dummies = pd.get_dummies(train[['component_type_1','component_type_2','component_type_3','component_type_4',
                                 'component_type_5','component_type_6','component_type_7','component_type_8',
                                 'forming_a','forming_x','end_a','end_x','end_a_1x','end_a_2x','end_x_1x',
                                 'end_x_2x','bracket_pricing']])

X_train = train.drop(['tube_assembly_id','supplier','quote_date','material_id','cost','component_id_1',
                     'component_id_2','component_id_3','component_id_4','component_id_5','component_id_6',
                     'component_id_7','component_id_8','component_type_1','component_type_2','component_type_3',
                     'component_type_4','component_type_5','component_type_6','component_type_7','component_type_8',
                     'forming_a','forming_x','end_a','end_x','end_a_1x','end_a_2x','end_x_1x','end_x_2x',
                     'bracket_pricing'],axis=1)
# X_train.select_dtypes(exclude=[np.float64])
X_train = pd.concat([X_train, dummies],axis=1).values
y_train = train['cost'].values
y_train = np.log1p(y_train)

In [17]:
X_train.shape

(30213, 151)

# Machine Learning Models

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
#Hit em with that Random Forest
forest_reg = RandomForestRegressor(n_estimators=100)
scores = cross_val_score(forest_reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
-scores.mean()

0.1131508988587981

In [19]:
#Train test split just to be sure
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25)
forest_reg.fit(X_train,y_train)
predictions = forest_reg.predict(X_test)
scores = mean_squared_error(predictions,y_test)
print(scores)

0.06453642100471638
